<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/WienerPathTensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
#@title Upgrade to TensorFlow nightly
#!pip install --upgrade tf-nightly-cpu

In [63]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

Python version
3.7.3 (v3.7.3:ef4ec6ed12, Mar 25 2019, 22:22:05) [MSC v.1916 64 bit (AMD64)]
Version info.
sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)


In [1]:
import tensorflow as tf
from scipy.special import erf
import numpy as np
import time
import math

In [2]:
SQRT_2 = tf.sqrt(tf.constant(2.,dtype=tf.float64))

def norminv(x):
    return SQRT_2*tf.math.erfinv(2*(x-0.5))

In [3]:
class UnivariateBrownianBridge():
  def __init__(self, number_time_steps):
    self.number_time_steps = number_time_steps

    self.left_index = np.zeros(number_time_steps, dtype=int)
    self.right_index = np.zeros(number_time_steps, dtype=int)
    self.bridge_index = np.zeros(number_time_steps, dtype=int)
    self.left_weight = np.zeros(number_time_steps)
    self.right_weight = np.zeros(number_time_steps)
    self.std_dev = np.zeros(number_time_steps)

    self._map = np.zeros(number_time_steps, dtype=int)

    self._map[-1] = 1
    self.bridge_index[0] = number_time_steps - 1
    self.std_dev[0] = math.sqrt(1.0 * number_time_steps)
    self.left_weight[0] = 0
    self.right_weight[0] = 0

    j=0
    for i in range(1,number_time_steps):
      while self._map[j] == True:
        j = j + 1
      k = j
      while self._map[k] == False:
        k = k + 1
      l = j+((k-1-j)>>1)
      self._map[l]=i
      self.bridge_index[i]=l
      self.left_index[i]=j
      self.right_index[i]=k
      self.left_weight[i]=(k-l)/(k+1-j)
      self.right_weight[i]=(1+l-j)/(k+1-j)
      self.std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
      j=k+1
      if j>=number_time_steps:
        j=0

    self.bridge_index_t = tf.constant(tf.convert_to_tensor(self.bridge_index,dtype=tf.int32))
    self.right_index_t = tf.constant(tf.convert_to_tensor(self.right_index,dtype=tf.int32))
    self.left_index_t = tf.constant(tf.convert_to_tensor(self.left_index,dtype=tf.int32))
    self.std_dev_t = tf.constant(tf.convert_to_tensor(self.std_dev))
    self.right_weight_t = tf.constant(tf.convert_to_tensor(self.right_weight))
    self.left_weight_t = tf.constant(tf.convert_to_tensor(self.left_weight))

  @tf.function(jit_compile=True, input_signature=(tf.TensorSpec(shape=(None,None,None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(), dtype=tf.bool),
                                                tf.TensorSpec(shape=(), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64)))
  def buildPath(z, increment, number_time_steps, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
    path = tf.TensorArray(dtype=tf.float64,size=number_time_steps+1)
    path = path.write(number_time_steps,std_dev[0]*z[0])
    path = path.write(0,0*z[0]);  
    j = 0
    k = 0
    l = 0
    i = 0
    for i in range(1,number_time_steps):
      j = left_index[i]
      k = right_index[i]
      l = bridge_index[i]
      path = path.write(l+1,left_weight[i] * path.read(j) + right_weight[i] * path.read(k+1) + std_dev[i] * z[i])

    if increment:
      for i in range(0,number_time_steps):
        path = path.write(i, path.read(i+1) - path.read(i))

    return path.stack()

  def path(self,z,increment):
    return UnivariateBrownianBridge.buildPath(z, increment, self.number_time_steps, self.left_index_t, self.right_index_t, self.bridge_index_t, self.left_weight_t, self.right_weight_t, self.std_dev_t)

In [4]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  2#@param {type:"integer"}
number =  7#@param {type:"integer"}

brownian = UnivariateBrownianBridge(dim)

x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
x = tf.transpose(x)
y = tf.reshape(x, shape=(dim,1,number))
z = norminv(y)

print(z)
print(brownian.path(z, False))
print(brownian.path(z, True))

tf.Tensor(
[[[ 0.          0.67448975 -0.67448975 -0.31863936  1.15034938
    0.31863936 -1.15034938]]

 [[ 0.         -0.67448975  0.67448975 -0.31863936  1.15034938
   -1.15034938  0.31863936]]], shape=(2, 1, 7), dtype=float64)
tf.Tensor(
[[[ 0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
    0.00000000e+00  0.00000000e+00 -0.00000000e+00]]

 [[ 0.00000000e+00 -2.44760480e-17  2.44760480e-17 -4.50624110e-01
    1.62683970e+00 -5.88107793e-01 -5.88107793e-01]]

 [[ 0.00000000e+00  9.53872552e-01 -9.53872552e-01 -4.50624110e-01
    1.62683970e+00  4.50624110e-01 -1.62683970e+00]]], shape=(3, 1, 7), dtype=float64)
tf.Tensor(
[[[ 0.00000000e+00 -2.44760480e-17  2.44760480e-17 -4.50624110e-01
    1.62683970e+00 -5.88107793e-01 -5.88107793e-01]]

 [[ 0.00000000e+00  9.53872552e-01 -9.53872552e-01  0.00000000e+00
    0.00000000e+00  1.03873190e+00 -1.03873190e+00]]

 [[ 0.00000000e+00  9.53872552e-01 -9.53872552e-01 -4.50624110e-01
    1.62683970e+00  4.50624110e-01 -1.62683

In [5]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  128#@param {type:"integer"}
number =  200000#@param {type:"integer"}

brownian = UnivariateBrownianBridge(dim)

time_start = time.time()
with tf.device("/cpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    x = tf.transpose(x)
    y = tf.reshape(x, shape=(dim,1,number))
    z = norminv(y)
    w = brownian.path(z, True)
time_end = time.time()
time_price_cpu = time_end - time_start
print("Runtime on CPU: ", time_price_cpu)

Runtime on CPU:  0.6260528564453125


In [7]:
dim =  128#@param {type:"integer"}
number =  200000#@param {type:"integer"}

time_start = time.time()

brownian = UnivariateBrownianBridge(dim)

with tf.profiler.experimental.Profile("logs/brownian"):
  with tf.device('/cpu'):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    x = tf.transpose(x)
    y = tf.reshape(x, shape=(dim,1,number))
    z = norminv(y)
   
    w = brownian.path(z, True)
time_end = time.time()
time_price_tpu = time_end - time_start
print("Time on a CPU: ", time_price_tpu)    

Time on a CPU:  0.6009738445281982


In [70]:
#!pip install -U tensorboard_plugin_profile

In [71]:
%load_ext tensorboard

In [72]:
S = 1
K = 1
sigma = 0.2
sdt = np.sqrt(1./dim)

p = tf.TensorArray(dtype=tf.float64,size=1)
p = p.write(0,S*tf.ones(shape=(number),dtype=tf.float64));  

In [73]:
brownian = UnivariateBrownianBridge(dim)

x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
x = tf.transpose(x)
y = tf.reshape(x, shape=(dim,1,number))
z = norminv(y)
w = brownian.path(z, True)

In [74]:
for i in range(dim):
    p = p.write(0, p.read(0) * (1. + sigma * sdt * w[i,0]))

In [75]:
x = p.read(0)

In [76]:
tf.math.reduce_mean(tf.math.maximum(x-1,0))

<tf.Tensor: shape=(), dtype=float64, numpy=0.07965782234994821>

In [77]:
w = brownian.path(z, False)

In [78]:
w[-1]

<tf.Tensor: shape=(1, 200000), dtype=float64, numpy=
array([[  0.        ,  10.791836  , -10.791836  , ...,   0.91186941,
        -32.00770287, -29.96743016]])>

In [79]:
def square_root_symmetric_matrix(A):
    w, v = tf.linalg.eigh(A)
    return tf.matmul(tf.matmul(v, tf.linalg.diag(tf.sqrt(w))),v)

In [138]:
class MultivariateBrownianBridge():
    def __init__(self, forwardCovarianceMatrices):
        self.forwardCovarianceMatrices = forwardCovarianceMatrices
        self.numberTimeSteps = len(forwardCovarianceMatrices)
        self.numberStates = len(forwardCovarianceMatrices[0])
        self.brownian = UnivariateBrownianBridge(self.numberTimeSteps)

        self.sqrtForwardCovarianceMatrices = tf.TensorArray(dtype=tf.float64, size=self.numberTimeSteps, clear_after_read=False)
        for i in range(self.numberTimeSteps):
            self.sqrtForwardCovarianceMatrices = self.sqrtForwardCovarianceMatrices.write(i, square_root_symmetric_matrix(self.forwardCovarianceMatrices[i]))

    def path(self, number):
        x = tf.math.sobol_sample(self.numberTimeSteps*self.numberStates,number,dtype=tf.dtypes.float64)
        x = tf.transpose(x)
        y = tf.reshape(x, shape=(self.numberTimeSteps,self.numberStates,number))
        z = norminv(y)

        w = self.brownian.path(z, True)
        
        path = tf.TensorArray(dtype=tf.float64,size=self.numberTimeSteps,clear_after_read=False)
        for i in range(len(z)):
            path = path.write(i,tf.matmul(self.sqrtForwardCovarianceMatrices.read(i), w[i]))

        return path.stack()

In [139]:
fwdCov = tf.ones(shape=(4,1,1), dtype=tf.float64)
multivariate_bridge = MultivariateBrownianBridge(fwdCov)

In [141]:
multivariate_bridge.path(7)

<tf.Tensor: shape=(4, 1, 7), dtype=float64, numpy=
array([[[ 0.        , -0.47693628,  0.47693628, -0.09332731,
          0.33692953,  0.39756484, -0.64116706]],

       [[ 0.        ,  0.47693628, -0.47693628, -0.54395142,
          1.96376923, -1.22927486, -0.19054295]],

       [[ 0.        ,  0.19755347, -0.19755347,  0.81341985,
         -0.22531206,  0.95980643, -1.54791422]],

       [[ 0.        ,  1.15142603, -1.15142603, -0.81341985,
          0.22531206,  0.50918232,  0.07892548]]])>

In [83]:
fm = tf.constant([[1.,0.5],[0.5,1.]], dtype=tf.float64)

e,v = tf.linalg.eigh(fm)

In [84]:
tf.sqrt(e)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.70710678, 1.22474487])>

In [85]:
tf.linalg.diag(tf.sqrt(e))

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[0.70710678, 0.        ],
       [0.        , 1.22474487]])>

In [86]:
e

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.5, 1.5])>

In [87]:
v

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[-0.70710678,  0.70710678],
       [ 0.70710678,  0.70710678]])>

In [88]:
    #return torch.mm(torch.mm(v, torch.diag(torch.sqrt(w[:]))), v.t())

In [89]:
tms = square_root_symmetric_matrix(fm)

In [90]:
tf.matmul(tms,tms)

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1. , 0.5],
       [0.5, 1. ]])>

In [91]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
times =  128#@param {type:"integer"}
states = 2
number =  2**16-1#@param {type:"integer"}

fwdCov = tf.ones(shape=(times,states,states), dtype=tf.float64)
multivariate_bridge = MultivariateBrownianBridge(fwdCov)

time_start = time.time()
with tf.device("/cpu:0"):
    w = multivariate_bridge.path(number)
time_end = time.time()
time_price_cpu = time_end - time_start
print("Runtime on CPU: ", time_price_cpu)

Runtime on CPU:  0.714928150177002


In [92]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
times =  8#@param {type:"integer"}
states = 2
number =  200000#@param {type:"integer"}

fwdCov = tf.ones(shape=(times,states,states), dtype=tf.float64)
multivariate_bridge = MultivariateBrownianBridge(fwdCov)

time_start = time.time()
with tf.profiler.experimental.Profile("logs/brownian"):
  with tf.device('/cpu'):
    w = multivariate_bridge.path(number)
time_end = time.time()
time_price_tpu = time_end - time_start
print("Time on a CPU: ", time_price_tpu)    

Time on a CPU:  0.29283809661865234


In [93]:
a = tf.Variable(tf.zeros(shape=(states,states),dtype=tf.float64))

In [94]:
rho = tf.constant(0.5,dtype=tf.float64)

In [95]:
rho = 0.6

In [96]:
rho

0.6

In [97]:
a[0,0] = 1

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "d:\Home\quant_analytics_torch\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3438, in run_code
    return outflag  File "<ipython-input-91-26a66e97b375>", line 11, in <module>
    w = multivariate_bridge.path(number)  File "<ipython-input-80-ec977b4bbb3e>", line 24, in path
    return path  File "d:\Home\quant_analytics_torch\venv\lib\site-packages\tensorflow\python\util\tf_should_use.py", line 249, in wrapped
    error_in_function=error_in_function)


TypeError: 'ResourceVariable' object does not support item assignment

In [98]:
correlation_matrix = [[1.,0.5],[0.5,1.]]

In [99]:
cor = tf.convert_to_tensor(correlation_matrix)
cor

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1. , 0.5],
       [0.5, 1. ]], dtype=float32)>

In [100]:
loss = tf.norm(cor)

In [101]:
rho = tf.Variable([[1,0]])

In [102]:
import tensorflow as tf

In [103]:
x = tf.Variable(0.5, dtype=tf.float64,name='x')
y = tf.Variable(1.5, dtype=tf.float64,name='y')
z = tf.Variable(2.5, dtype=tf.float64,name='z')

with tf.GradientTape(persistent=True) as g:
    a = tf.TensorArray(size=0, dynamic_size=True, dtype=tf.float64)
    m = tf.TensorArray(size=0, dynamic_size=True, dtype=tf.float64)
    a = a.write(0, x)
    a = a.write(1, y)    
    a = a.write(2, z)        
    b = a.stack()
    m = m.write(0, b)
    m = m.write(1, b)    
    c = m.stack()
    print(c)
    loss = tf.norm(c)
    print(loss)

# Loop over all watched variables
for it in g.watched_variables():
    dy_dx = g.gradient(loss, it)
    print(it.name)
    print(dy_dx)

print(g.watched_variables())

del g

tf.Tensor(
[[0.5 1.5 2.5]
 [0.5 1.5 2.5]], shape=(2, 3), dtype=float64)
tf.Tensor(4.183300132670378, shape=(), dtype=float64)
x:0
tf.Tensor(0.23904572186687872, shape=(), dtype=float64)
y:0
tf.Tensor(0.7171371656006362, shape=(), dtype=float64)
z:0
tf.Tensor(1.1952286093343936, shape=(), dtype=float64)
(<tf.Variable 'x:0' shape=() dtype=float64, numpy=0.5>, <tf.Variable 'y:0' shape=() dtype=float64, numpy=1.5>, <tf.Variable 'z:0' shape=() dtype=float64, numpy=2.5>)


In [104]:
print(dy_dx)

tf.Tensor(1.1952286093343936, shape=(), dtype=float64)


Price a spread option

In [175]:
EPSILON = tf.constant(0.001,dtype=tf.float64)
DELTA = tf.constant(EPSILON/tf.sqrt(2.),dtype=tf.float64)
BOUNDARY = tf.constant(EPSILON*tf.sqrt(2.),dtype=tf.float64)

def phi_smooth(x,y):
    return (x + y + DELTA + (x - y) ** 2 / DELTA / 4.)/2.

def max_if(x,y):
    return tf.where(tf.abs(x-y)>BOUNDARY,tf.maximum(x,y),phi_smooth(x,y))

TypeError: x and y must have the same dtype, got tf.float64 != tf.float32

In [203]:
times =  1#@param {type:"integer"}
states = 2
number =  2**20-1#@param {type:"integer"}

f_1 = tf.Variable(100., dtype=tf.float64,name='f_1')
f_2 = tf.Variable(100., dtype=tf.float64,name='f_2')

sigma_1 = tf.Variable(0.2, dtype=tf.float64,name='s_1')
sigma_2 = tf.Variable(0.2, dtype=tf.float64,name='s_2')

rho = tf.Variable(0.2, dtype=tf.float64, name='rho')

time_start = time.time()

with tf.GradientTape(persistent=True) as g:
    fwdCovs = tf.TensorArray(size=0, dynamic_size=True, dtype=tf.float64)

    fwdCov = tf.TensorArray(size=0, dynamic_size=True, dtype=tf.float64)

    tmp_1 = tf.TensorArray(size=0, dynamic_size=True, dtype=tf.float64)
    tmp_1 = tmp_1.write(0, sigma_1*sigma_1)
    tmp_1 = tmp_1.write(1, sigma_1*sigma_2*rho)    
    fwdCov = fwdCov.write(0, tmp_1.stack())

    tmp_2 = tf.TensorArray(size=0, dynamic_size=True, dtype=tf.float64)
    tmp_2 = tmp_2.write(0, sigma_1*sigma_2*rho)
    tmp_2 = tmp_2.write(1, sigma_2*sigma_2)    
    fwdCov = fwdCov.write(1, tmp_2.stack())

    fwdCovs = fwdCovs.write(0, fwdCov.stack())

    print(fwdCovs.stack())

    multivariate_bridge = MultivariateBrownianBridge(fwdCovs.stack())

    w = multivariate_bridge.path(number)

    s_1 = f_1 * (1. + w[0,0,:])
    s_2 = f_2 * (1. + w[0,1,:])    

    v = s_1 - s_2

    #print(w)

    #print(s_1)
    #print(s_2)    

    #print(v)

    pv = tf.reduce_mean(tf.maximum(s_1 + s_2 - 200.,0.))
    print(pv)
    #pv = tf.reduce_mean(max_if(s_1 - s_2,tf.constant(0.,dtype=tf.float64)))

time_end = time.time()
time_price_tpu = time_end - time_start
print("Time on a CPU: ", time_price_tpu)    

time_start = time.time()

dy_dx = g.gradient(pv, g.watched_variables())
print(dy_dx)

time_end = time.time()
time_price_tpu = time_end - time_start
print("Time on a CPU: ", time_price_tpu)


del g


tf.Tensor(
[[[0.04  0.008]
  [0.008 0.04 ]]], shape=(1, 2, 2), dtype=float64)
tf.Tensor(12.360587602925008, shape=(), dtype=float64)
Time on a CPU:  0.08700180053710938
(<tf.Tensor: shape=(), dtype=float64, numpy=0.562287501143537>, <tf.Tensor: shape=(), dtype=float64, numpy=0.5622901699409071>, <tf.Tensor: shape=(), dtype=float64, numpy=30.900719259160358>, <tf.Tensor: shape=(), dtype=float64, numpy=30.90221875546465>, <tf.Tensor: shape=(), dtype=float64, numpy=5.150244834552082>)
Time on a CPU:  0.03802943229675293
